<a href="https://colab.research.google.com/github/redball1017/Colab-Hacks-CN/blob/master/Colab%20RDP/Colab%20RDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Colab RDP** : 远程桌面到 Colab 实例

> **警告：不适用于加密货币挖矿<br></br>** 
>**为什么我无法使用 T4 GPU 等硬件资源？** 最好的可用硬件优先用于交互使用 Colaboratory 的用户，而不是用于长时间运行的计算。 使用 Colaboratory 进行长时间运行计算的用户可能会暂时受限于提供给他们的硬件类型和/或硬件可以使用的持续时间。 我们鼓励具有高计算需求的用户将 Colaboratory 的 UI 与本地运行时一起使用。 请注意，完全禁止使用 Colaboratory 进行加密货币挖掘，并可能导致完全禁止使用 Colab。

Google Colab 可以为免费用户提供具有 12GB RAM 和 GPU 的 12 小时（最长）的实例。 任何人都可以使用它来执行繁重的任务。

要使用其他类似的笔记本，请使用我的Repo **[Colab Hacks](https://github.com/redball1017/Colab-Hacks-CN)**

In [ ]:
#@title **创建用户**
#@markdown 输入用户名和密码

import os

username = "user" #@param {type:"string"}
password = "root" #@param {type:"string"}

print("正在创建用户")

# 用户创建程序
os.system(f"useradd -m {username}")

# 将用户添加到sudo组
os.system(f"adduser {username} sudo")
    
# 设置用户的密码为 'root'
os.system(f"echo '{username}:{password}' | sudo chpasswd")

# 将默认终端程序从sh更改到bash
os.system("sed -i 's/\/bin\/sh/\/bin\/bash/g' /etc/passwd")

print(f"创建并配置用户名“{username}”和密码“{password}”的用户")

In [ ]:
#@title **RDP**
#@markdown  安装需要4-5分钟

import os
import subprocess

#@markdown  访问 http://remotedesktop.google.com/headless 并在 Authentication 后复制命令

CRP = "" #@param {type:"string"}

#@markdown 输入 Pin （多于或等于 6 位数字）
Pin = 123456 #@param {type: "integer"}

#@markdown RDP 中的自动启动笔记本
Autostart = False #@param {type: "boolean"}


class CRD:
    def __init__(self, user):
        os.system("apt update")
        self.installCRD()
        self.installDesktopEnvironment()
        self.installGoogleChorme()
        self.finish(user)
        print("\nRDP 创建成功移至 https://remotedesktop.google.com/access")

    @staticmethod
    def installCRD():
        print("安装 Chrome 远程桌面")
        subprocess.run(['wget', 'https://dl.google.com/linux/direct/chrome-remote-desktop_current_amd64.deb'], stdout=subprocess.PIPE)
        subprocess.run(['dpkg', '--install', 'chrome-remote-desktop_current_amd64.deb'], stdout=subprocess.PIPE)
        subprocess.run(['apt', 'install', '--assume-yes', '--fix-broken'], stdout=subprocess.PIPE)

    @staticmethod
    def installDesktopEnvironment():
        print("Installing Desktop Environment")
        os.system("export DEBIAN_FRONTEND=noninteractive")
        os.system("apt install --assume-yes xfce4 desktop-base xfce4-terminal")
        os.system("bash -c 'echo \"exec /etc/X11/Xsession /usr/bin/xfce4-session\" > /etc/chrome-remote-desktop-session'")
        os.system("apt remove --assume-yes gnome-terminal")
        os.system("apt install --assume-yes xscreensaver")
        os.system("systemctl disable lightdm.service")

    @staticmethod
    def installGoogleChorme():
        print("正在安装Google Chrome")
        subprocess.run(["wget", "https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb"], stdout=subprocess.PIPE)
        subprocess.run(["dpkg", "--install", "google-chrome-stable_current_amd64.deb"], stdout=subprocess.PIPE)
        subprocess.run(['apt', 'install', '--assume-yes', '--fix-broken'], stdout=subprocess.PIPE)

    @staticmethod
    def finish(user):
        print("搞定")
        if Autostart:
            os.makedirs(f"/home/{user}/.config/autostart", exist_ok=True)
            link = "https://colab.research.google.com/github/PradyumnaKrishna/Colab-Hacks/blob/master/Colab%20RDP/Colab%20RDP.ipynb"
            colab_autostart = """[Desktop Entry]
Type=Application
Name=Colab
Exec=sh -c "sensible-browser {}"
Icon=
Comment=在会话登录时打开预定义的笔记本.
X-GNOME-Autostart-enabled=true""".format(link)
            with open(f"/home/{user}/.config/autostart/colab.desktop", "w") as f:
                f.write(colab_autostart)
            os.system(f"chmod +x /home/{user}/.config/autostart/colab.desktop")
            os.system(f"chown {user}:{user} /home/{user}/.config")

        os.system(f"adduser {user} chrome-remote-desktop")
        command = f"{CRP} --pin={Pin}"
        os.system(f"su - {user} -c '{command}'")
        os.system("service chrome-remote-desktop start")
        

        print("成功完成")


try:
    if CRP == "":
        print("请输入来自给定链接的验证码")
    elif len(str(Pin)) < 6:
        print("输入大于或等于 6 位数字的PIN密码")
    else:
        CRD(username)
except NameError as e:
    print("'username' 变量未找到，首先创建一个用户")

In [ ]:
%env USE_AUTH_EPHEM=0

#@title **谷歌云端硬盘装载**
#@markdown Google Drive 可用作文件的持久硬盘.<br>

#@markdown **选择一种方法（GDFuse 推荐）**
mount_method = "GDFuse" #@param ["GDFuse", "Native"]


#@markdown **GDFuse 的选项** <br>
#@markdown - 访问 https://github.com/astrada/google-drive-ocamlfuse/wiki/Team-Drives
label = "default" #@param {type:"string"}
mount_team_drive = False #@param {type:"boolean"}
force_mount = False #@param {type:"boolean"}

import os
import subprocess


class Drive():
    creds = {}
    mountpoint = ""
    deps = False
    
    debug = False

    def __init__(self, mountpoint="/content/drives", debug=False):
        os.makedirs(mountpoint, exist_ok=True)
        self.mountpoint = mountpoint
        self.debug = debug

    def _mount_gdfuse(self, mount_dir):
        os.makedirs(mount_dir, exist_ok=True)

        subprocess.run(
            ['google-drive-ocamlfuse',
             '-o',
             'allow_other',
             '-label',
             label,
             mount_dir,
            ]
        )

        print(f"驱动器安装在 {mount_dir}。 如果您收到输入/输出错误，则 `team_drive_id` 可能是错误的或无法访问.")

    def _unmount_gdfuse(self, mount_dir):
        subprocess.run(
            ['fusermount',
             '-u',
             mount_dir,
            ]
        )
        os.rmdir(mount_dir)

    def auth(self):
        from google.colab import auth
        from oauth2client.client import GoogleCredentials

        auth.authenticate_user()
        
        creds = GoogleCredentials.get_application_default()
        self.creds = {
            "id": creds.client_id,
            "secret": creds.client_secret
        }

    def gdfuse(self, label, mound_team_drive=False, force_mount=False):
        import getpass

        if not self.creds:
            self.auth()

        if not self.deps:
            print("Installing google-drive-ocamlfuse")
            subprocess.run(['apt', 'install', 'software-properties-common python-software-properties module-init-tools', '-y'])
            subprocess.run(['add-apt-repository', 'ppa:alessandro-strada/ppa', '-y'])
            subprocess.run(['apt', 'update'])
            subprocess.run(['apt', 'install', '--assume-yes', 'google-drive-ocamlfuse'])
            self.deps = True

        base_dir = '/root/.gdfuse'
        config_dir = f'{base_dir}/{label}'
        mount_dir = f"{self.mountpoint}/{label}"

        if force_mount and os.path.exists(mount_dir):
            self._unmount_gdfuse(mount_dir)
        elif os.path.exists(mount_dir):
            print("驱动器已安装")
            return

        if not os.path.exists(config_dir) or force_mount:
            print(f"请在网络浏览器中打开以下 URL：https://accounts.google.com/o/oauth2/auth?client_id={self.creds['id']}&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2 .0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force")
            vcode = getpass.getpass("Enter the Auth Code: ")

            subprocess.run(
              ['google-drive-ocamlfuse',
                '-headless',
                '-id',
                self.creds['id'],
                '-secret',
                self.creds['secret'],
                '-label',
                label,
              ],
                text=True,
                input=vcode
            )

        if mount_team_drive:
            team_drive_id = input("输入共享云端硬盘 ID: ")

            subprocess.run(
                ['sed',
                 '-i',
                 f's/team_drive_id=.*$/team_drive_id={team_drive_id}/g',
                 f'{config_dir}/config'
                ]
            )
        else:
            subprocess.run(
                ['sed',
                 '-i',
                 f's/team_drive_id=.*$/team_drive_id=/g',
                 f'{config_dir}/config'
                ]
            )

        self._mount_gdfuse(mount_dir)


    def native(self):
        from google.colab import drive
        mount_dir = f"{self.mountpoint}/Native"
        drive.mount(mount_dir)

if 'drive' not in globals():
    try:
        drive = Drive(f"/home/{username}/drives")
    except NameError:
        drive = Drive('/content/drives')

if mount_method == "Native":
    drive.native()
if mount_method == "GDFuse":
    drive.gdfuse(label, mount_team_drive, force_mount)


In [ ]:
#@title **SSH**

! pip install colab_ssh --upgrade &> /dev/null

#@markdown 选择一种方法（Argo推荐）
ssh_method = "Agro" #@param ["Agro", "Ngrok"]


#@markdown 从 https://dashboard.ngrok.com/auth 复制 authtoken（仅适用于 ngrok）
ngrokRegion = "us" #@param ["us", "eu", "ap", "au", "sa", "jp", "in"]

def runAgro():
    from colab_ssh import launch_ssh_cloudflared
    launch_ssh_cloudflared(password=password)

def runNgrok():
    from colab_ssh import launch_ssh
    from IPython.display import clear_output

    import getpass
    ngrokToken = getpass.getpass("输入 ngrokToken: ")

    launch_ssh(ngrokToken, password, region=ngrokRegion)
    clear_output()

    print("ssh", user, end='@')
    ! curl -s http://localhost:4040/api/tunnels | python3 -c \
            "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'][6:].replace(':', ' -p '))"

try:
    user = username
    password = password
except NameError:
    print("未找到用户，使用用户名和密码“root”")
    user='root'
    password='root'


if ssh_method == "Agro":
    runAgro()
if ssh_method == "Ngrok":
    runNgrok()

In [ ]:
#@title **Colab 关闭**

#@markdown 杀死 NGROK 隧道
NGROK = False #@param {type:'boolean'}

#@markdown To Sleep Colab
Sleep = True #@param {type:'boolean'}

if NGROK:
    ! killall ngrok

if Sleep:
    from time import sleep
    sleep(43200)